# Agente LangChain

Este notebook implementa un agente que recopila noticias de las Fiestas Patrias y las procesa usando un LLM en OCI.

## Funcionalidades

📰 Recopila noticias sobre las Fiestas Patrias

🤖 Procesa con un LLM vía OCI

🔍 Búsqueda inteligente con LangChain

## Recopila noticias sobre las Fiestas Patrias

In [1]:
!pip install tavily-python
!pip install -U langchain-community langchain-core oci

In [74]:
import requests
from langchain_core.tools import tool
from typing import Dict, List
import re
from tavily import TavilyClient

CHILE_FIESTAS_DOMAINS = [
    "gob.cl", "minsal.cl", "minagri.gob.cl", "diariooficial.interior.gob.cl",
    "munistgo.cl", "muninunoa.cl", "munilascondes.cl", "cooperativa.cl",
    "latercera.com", "emol.com", "biobiochile.cl", "24horas.cl"
]
@tool
def get_fiestas_patrias_comprehensive_news(pregunta:str="Programa, feriados y fondas de Fiestas Patrias en Santiago") -> str:
    """Devuelve info/noticias de Fiestas Patrias en Chile (fondas, ramadas, Parada Militar, feriados, normas)."""
    
    client = TavilyClient("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") #es la API de dev
    response = client.search(
    query=pregunta,
    #include_domains=CHILE_FIESTAS_DOMAINS,
    #topic="news",
    #days=45,
    #max_results=10
    )
    return response

## Configuración de la autenticación del SDK de OCI

In [59]:
# crea carpeta y permisos
!mkdir -p /home/datascience/.oci

In [4]:
# Ver tu HOME y listar (incluye ocultos)
!echo $HOME
!ls -la $HOME | head -n 30

/home/datascience
total 26214504
drwxr-xr-x. 13 datascience users         4096 Sep  8 02:22 .
drwxr-xr-x.  1 root        root          4096 Aug 10 18:52 ..
-rw-------.  1 nobody      nobody 26843545600 Sep  8 02:14 .__nb_swap__
drwxr-xr-x.  6 datascience users         4096 Sep  8 00:05 .cache
drwxr-xr-x.  3 datascience users         4096 Sep  8 00:02 .config
drwxr-xr-x.  2 datascience users         4096 Sep  8 02:17 .ipynb_checkpoints
drwxr-xr-x.  3 datascience users         4096 Sep  8 00:05 .ipython
drwxr-xr-x.  3 datascience users         4096 Sep  8 00:02 .local
drwxr-x---.  2 datascience users         4096 Sep  8 00:02 .mnt
drwxr-xr-x.  2 datascience users         4096 Sep  8 02:18 .oci
drwxr-x---.  3 nobody      nobody        4096 Sep  8 00:02 .rclone
drwxr-xr-x.  2 datascience users         4096 Sep  8 00:02 .runtime-config-logs
-rw-r--r--.  1 datascience users        28759 Sep  8 02:22 Untitled3.ipynb
-rw-r--r--.  1 datascience users           72 Sep  8 01:41 getting_started.ip

In [8]:
!mkdir -p ~/.oci
!ls -la ~/.oci


total 8
drwxr-xr-x.  2 datascience users 4096 Sep  8 02:18 .
drwxr-xr-x. 13 datascience users 4096 Sep  8 02:26 ..


In [16]:
!mv ~/oci_api_key.pem ~/.oci/oci_api_key.pem
!chmod 600 ~/.oci/oci_api_key.pem
!ls -la ~/.oci

total 16
drwxr-xr-x.  2 datascience users 4096 Sep  8 02:38 .
drwxr-xr-x. 13 datascience users 4096 Sep  8 02:38 ..
-rw-r--r--.  1 datascience users  371 Sep  8 02:36 config
-rw-------.  1 datascience users 1715 Sep  8 02:38 oci_api_key.pem


In [24]:
%%bash
cat > ~/.oci/config <<'CFG'
[DEFAULT]
user=ocid1.user.oc1..aaaaaaaao3vm22tuwkzluqdgeoka6uq7xaix2zlmpnmwrj27bcrdsufdkuha
tenancy=ocid1.tenancy.oc1..aaaaaaaaoi6b5sxlv4z773boczybqz3h2vspvvru42jysvizl77lky22ijaq
region=us-chicago-1
fingerprint=f6:ec:66:50:04:b4:be:ca:55:c6:21:03:16:73:67:25
key_file=/home/datascience/.oci/oci_api_key.pem
CFG

echo "Config creado en ~/.oci/config"
cat ~/.oci/config | sed 's/fingerprint=.*/fingerprint=<oculto>/'

Config creado en ~/.oci/config
[DEFAULT]
user=ocid1.user.oc1..aaaaaaaao3vm22tuwkzluqdgeoka6uq7xaix2zlmpnmwrj27bcrdsufdkuha
tenancy=ocid1.tenancy.oc1..aaaaaaaaoi6b5sxlv4z773boczybqz3h2vspvvru42jysvizl77lky22ijaq
region=us-chicago-1
fingerprint=<oculto>
key_file=/home/datascience/.oci/oci_api_key.pem


In [68]:
# Quita posibles finales de línea de Windows (CRLF)
!sed -i 's/\r$//' ~/.oci/config

# mostrar
!sed -n '1,200p' ~/.oci/config

[DEFAULT]
user=ocid1.user.oc1..aaaaaaaao3vm22tuwkzluqdgeoka6uq7xaix2zlmpnmwrj27bcrdsufdkuha
tenancy=ocid1.tenancy.oc1..aaaaaaaaoi6b5sxlv4z773boczybqz3h2vspvvru42jysvizl77lky22ijaq
region=us-chicago-1
fingerprint=f6:ec:66:50:04:b4:be:ca:55:c6:21:03:16:73:67:25
key_file=/home/datascience/.oci/oci_api_key.pem


## 🤖 Creación del Agente LangChain

In [77]:
from langchain_core.messages import HumanMessage
from langchain_community.chat_models import ChatOCIGenAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.prompts import PromptTemplate

# Configura tu endpoint y compartimento
REGION = "us-chicago-1"  # cambia si usas otra región para GenAI
ENDPOINT = f"https://inference.generativeai.{REGION}.oci.oraclecloud.com"

llm = ChatOCIGenAI(
  model_id="meta.llama-4-maverick-17b-128e-instruct-fp8",
  service_endpoint="https://inference.generativeai.us-chicago-1.oci.oraclecloud.com",
  compartment_id="ocid1.compartment.oc1..aaaaaaaawy5gpq67r3rxc7yf6yn7oesvvgr56dqg7o5y3f5ndzt2mddtj46a",
  provider="meta",
  model_kwargs={
    "temperature": 0.3, 
    "max_tokens": 800,   
    "top_p": 0.8,  
    "frequency_penalty": 0,
    "presence_penalty": 0,
  },
  auth_type="API_KEY",
  auth_profile="DEFAULT"
)

tools = [get_fiestas_patrias_comprehensive_news]

react_prompt_template = """Eres un asistente especializado en **Fiestas Patrias de Chile**.
Responde en español de Chile y usa las herramientas disponibles cuando sea útil.

REGLAS IMPORTANTES:
- Tras usar una herramienta, **resume con tus palabras** los hallazgos (no pegues el JSON).
- Prioriza información **reciente y oficial** (Gobierno, municipalidades, medios chilenos).
- Incluye **fechas, lugares, horarios** y **enlaces** útiles cuando existan.
- Si hay discrepancias entre fuentes, indícalas brevemente.

Herramientas disponibles:
{tools}

Usa EXACTAMENTE este formato:

Question: la pregunta a responder
Thought: explica qué harás
Action: una de [{tool_names}]
Action Input: el input para la acción (o "" si no aplica)
Observation: resultado de la acción
Thought: analiza y sintetiza
Final Answer: respuesta clara y útil en español de Chile

Comienza.

Question: {input}
Thought:{agent_scratchpad}"""

prompt = PromptTemplate.from_template(react_prompt_template)
agent = create_react_agent(llm, tools, prompt)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    max_iterations=3,
    stream_runnable=False,
    handle_parsing_errors=True
)
pregunta = "cuándo son las fechas patrias en Chile?"
respuesta = agent_executor.invoke({"input": pregunta})

pregunta = "¿Cuáles son las mejores fondas en Santiago y cuáles son los horarios? ¿Habrá Parada Militar?"
respuesta = agent_executor.invoke({"input": pregunta})




> Entering new AgentExecutor chain...
Thought: Para responder a la pregunta sobre cuándo son las fechas patrias en Chile, necesito obtener información oficial y reciente sobre los feriados de Fiestas Patrias en Chile.

Action: get_fiestas_patrias_comprehensive_news
Action Input: fechas de Fiestas Patrias en Chile
{'query': 'fechas de Fiestas Patrias en Chile\n', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://chile.travel/eventos/fiestas-patrias/', 'title': 'Fiestas Patrias - Chile Travel', 'content': 'Todos los años, el 18 de septiembre, Chile celebra grandiosamente las Fiestas Patrias para conmemorar el inicio del proceso de Independencia de la Corona Española. Conocidas comúnmente como “el dieciocho” (en alusión a la fecha), la mayoría de las celebraciones más grandes ocurren alrededor de los días 18 y 19, pero el período de festividad puede durar hasta una semana. La gran mayoría de los chilenos celebra esta festividad en asados y “fondas”,

In [78]:
from IPython.display import Markdown, display
import json

# 1) Toma la salida del agente y conviértela a texto de forma segura
def _to_text(x):
    if isinstance(x, dict):
        # LangChain AgentExecutor suele devolver {"output": "..."}
        return x.get("output") or json.dumps(x, ensure_ascii=False, indent=2)
    return str(x)

insumos = _to_text(respuesta)   # <--- usa la variable 'respuesta' que ya tienes del agente

# 2) Prompt de análisis/síntesis para Fiestas Patrias (Chile)
analysis_prompt = f"""
Eres un analista chileno especializado en **Fiestas Patrias**.
Usa solo los datos entregados más abajo para responder en **español de Chile**, claro y útil.

DATOS RECOLECTADOS (pueden incluir JSON):
{insumos}

TAREAS:
1) Responde directamente a la pregunta: "{pregunta}".
2) Extrae y organiza lo más importante (si está disponible): **fechas** (18–19 de septiembre), **fondas/ramadas** por comuna,
   **lugar exacto**, **horarios**, **precio/entradas**, **requisitos**, y **enlaces oficiales**.
3) Indica si habrá **Parada Militar** (fecha, horario y lugar) y cualquier impacto (cortes, accesos, transporte) si está en los datos.
4) Si hay discrepancias entre fuentes, menciónalas brevemente.

FORMATO (Markdown):
## Resumen
- 3–5 líneas con lo esencial.

## Fondas y ramadas (si hay datos)
- **Comuna – Evento**: lugar | fechas | horarios | entradas | enlace

## Parada Militar (si aplica)
- fecha | hora | lugar | enlaces

## Fuentes
- Lista de URLs citadas (solo si aparecen en los datos).

REGLAS:
- **No inventes** datos ni enlaces. Si algo no está en los datos, escribe: *No disponible*.
- Prioriza información **reciente y oficial** (gob.cl, municipalidades, medios chilenos).
"""

# 3) Invoca la LLM (no streaming) y muestra en Markdown
analysis_response = llm.invoke(analysis_prompt)
display(Markdown(analysis_response.content))

## Resumen
- Las fondas en Santiago para Fiestas Patrias 2025 tienen varias opciones, como "La Yein Fonda 2025" en Parque Ciudad Empresarial de Huechuraba.
- Es recomendable verificar detalles específicos en fuentes como https://www.fiestascostumbristas.cl/2025/08/02/fondas-santiago-2025/ y otras.
- La información sobre la Parada Militar no está disponible en los datos obtenidos.
- Se sugiere buscar en fuentes oficiales del Gobierno o medios chilenos para obtener información actualizada.

## Fondas y ramadas
- **Huechuraba – La Yein Fonda 2025**: Parque Ciudad Empresarial | 17 al 20 de septiembre | *No disponible* | *No disponible* | https://www.fiestascostumbristas.cl/2025/08/02/fondas-santiago-2025/
- Otras fondas en la Región Metropolitana tienen diferentes horarios y precios. Para más detalles, revisar fuentes como https://www.latercera.com/servicios/noticia/donde-celebrar-fiestas-patrias-estas-son-las-fondas-de-la-rm-con-su-programacion-de-artistas-y-precios-de-las-entradas/ y https://chileestuyo.cl/fondas-en-santiago-conoce-las-mejores-para-celebrar-fiestas-patrias/

## Parada Militar
- *No disponible*. Se sugiere buscar en fuentes oficiales del Gobierno o medios chilenos para obtener información actualizada sobre este evento.

## Fuentes
- https://www.fiestascostumbristas.cl/2025/08/02/fondas-santiago-2025/
- https://www.latercera.com/servicios/noticia/donde-celebrar-fiestas-patrias-estas-son-las-fondas-de-la-rm-con-su-programacion-de-artistas-y-precios-de-las-entradas/
- https://chileestuyo.cl/fondas-en-santiago-conoce-las-mejores-para-celebrar-fiestas-patrias/
- https://www.13.cl/programas/servicios-13/actualidad/para-disfrutar-gratis-las-fiestas-patrias-estas-son-las-fondas